In [1]:
# Kaggle instalaltion in Colab # https://www.kaggle.com/general/74235
!pip install kaggle

In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 9.6MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 37.5MB/s 


In [ ]:
from google.colab import files
files.upload()

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c commonlitreadabilityprize

  0% 0.00/6.79k [00:00<?, ?B/s]
100% 6.79k/6.79k [00:00<00:00, 11.4MB/s]
  0% 0.00/108 [00:00<?, ?B/s]
100% 108/108 [00:00<00:00, 95.4kB/s]
  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 77.0MB/s]


In [6]:
!mkdir input
!unzip train.csv.zip -d input

Archive:  train.csv.zip
  inflating: input/train.csv         


In [7]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [8]:
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')

data = pd.concat([train_data,test_data])
data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n',''))

text  = '\n'.join(data.excerpt.tolist())

with open('input/text.txt','w') as f:
    f.write(text)

In [9]:
model_name = 'roberta-base'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('input/clrp_roberta_base')

('input/clrp_roberta_base/tokenizer_config.json',
 'input/clrp_roberta_base/special_tokens_map.json',
 'input/clrp_roberta_base/vocab.json',
 'input/clrp_roberta_base/merges.txt',
 'input/clrp_roberta_base/added_tokens.json',
 'input/clrp_roberta_base/tokenizer.json')

In [10]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="input/text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="input/text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="input/clrp_roberta_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [11]:
trainer.train()
trainer.save_model(f'input/clrp_roberta_base')

***** Running training *****
  Num examples = 2841
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 890


Step,Training Loss,Validation Loss
200,No log,1.425119
400,No log,1.376833
600,1.553600,1.336839
800,1.553600,1.299120


***** Running Evaluation *****
  Num examples = 2841
  Batch size = 16
Saving model checkpoint to input/clrp_roberta_base_chk/checkpoint-200
Configuration saved in input/clrp_roberta_base_chk/checkpoint-200/config.json
Model weights saved in input/clrp_roberta_base_chk/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2841
  Batch size = 16
Saving model checkpoint to input/clrp_roberta_base_chk/checkpoint-400
Configuration saved in input/clrp_roberta_base_chk/checkpoint-400/config.json
Model weights saved in input/clrp_roberta_base_chk/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2841
  Batch size = 16
Saving model checkpoint to input/clrp_roberta_base_chk/checkpoint-600
Configuration saved in input/clrp_roberta_base_chk/checkpoint-600/config.json
Model weights saved in input/clrp_roberta_base_chk/checkpoint-600/pytorch_model.bin
Deleting older checkpoint [input/clrp_roberta_base_chk/checkpoint-200] due to args.save_tot

In [12]:
!zip -r /content/file.zip /content/input/clrp_roberta_base/

  adding: content/input/clrp_roberta_base/ (stored 0%)
  adding: content/input/clrp_roberta_base/config.json (deflated 49%)
  adding: content/input/clrp_roberta_base/tokenizer_config.json (deflated 47%)
  adding: content/input/clrp_roberta_base/pytorch_model.bin (deflated 7%)
  adding: content/input/clrp_roberta_base/training_args.bin (deflated 48%)
  adding: content/input/clrp_roberta_base/special_tokens_map.json (deflated 50%)
  adding: content/input/clrp_roberta_base/tokenizer.json (deflated 59%)
  adding: content/input/clrp_roberta_base/vocab.json (deflated 59%)
  adding: content/input/clrp_roberta_base/merges.txt (deflated 53%)


In [13]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>